In [ ]:
using BenchmarkTools
using CSV
using DataFrames
using Printf
using StatsBase

include("../src/Kraft.jl")

In [ ]:
const elements = readlines("genes.txt")[2:end]

const element_values = randn(length(elements))

const element_x_sample = DataFrame(
    Symbol("Element") => elements,
    Symbol("Sample Constant") => fill(
        1 / length(elements),
        length(elements),
    ),
    Symbol("Sample Normal") => element_values,
    Symbol("Sample Normal x 10") => element_values * 10,
)

CSV.write("gene_x_sample.tsv", element_x_sample, delim = '\t')

const random_set_elements = sample(
    elements,
    100,
    replace = false,
)

const set_elements = Kraft.read_gmt([
    "h.all.v6.2.symbols.gmt",
])

const vector_01 = Kraft.make_vector_01(
    elements,
    random_set_elements,
)

;

In [ ]:
@benchmark Kraft.compute_set_enrichment(
# Kraft.compute_set_enrichment(
    element_values,
    elements,
    vector_01,
)

In [ ]:
@benchmark Kraft.compute_set_enrichment(
# Kraft.compute_set_enrichment(
    element_values,
    elements,
    random_set_elements,
)

In [ ]:
Kraft.plot_set_enrichment(
    element_values,
    elements,
    random_set_elements;
    title1_text = "Title",
    title2_text = "Description or any other text go here",
    element_value_name = "Element<br>Value<br>Metric",
)

In [ ]:
benchmark_result = @benchmark Kraft.compute_set_enrichment(
    element_values,
    elements,
    set_elements,
)

@printf "%.2f ms / set" minimum(benchmark_result.times) / 1e6 / length(set_elements)

# Kraft.compute_set_enrichment(
#     element_values,
#     elements,
#     set_elements,
# )

In [ ]:
@benchmark set_x_sample = Kraft.compute_set_enrichment(
# Kraft.compute_set_enrichment(
    element_x_sample,
    set_elements,
    "ks",
)

In [ ]:
output_directory_path = "gsea_output"

set_x_sample = Kraft.gsea(
    "gene_x_sample.tsv",
    [
        "h.all.v6.2.symbols.gmt",
    ],
    output_directory_path;
    sample_normalization_method = "-0-",
    gene_sets = nothing,
    n_required_gene_set_element = 1,
    weight = nothing,
    statistic = "ks",
)

CSV.read(joinpath(
    output_directory_path,
    "gene_set_x_sample.tsv",
))